# Trace report

In [ ]:
trace_file = None

In [ ]:
# Initialize GAPID environment
from helpers import gapid_env
from helpers import image_formats
from helpers import jupyter_helpers
from helpers import replay
from helpers import trace as gapid_trace
from helpers import vulkan


from importlib import reload

def reload_helpers():
    reload(gapid_env)
    reload(image_formats)
    reload(jupyter_helpers)
    reload(replay)
    reload(gapid_trace)
    reload(vulkan)

%matplotlib inline
#%matplotlib notebook

import os
import matplotlib
import matplotlib.pyplot as plt
import numpy
import math
from ipyfilechooser import FileChooser
from io import BytesIO
import imageio
import ipywidgets
from IPython.display import Markdown
from types import SimpleNamespace

# Set a good default plot size
matplotlib.rcParams['figure.figsize'] = [32, 18]

display(Markdown("""
## Initializing GAPID environment
"""))

pathdir = globals()['_dh'][0]
env = gapid_env.gapid_env(os.path.normpath(os.path.join(pathdir, "..", "build", "RelWithDebInfo")))
env.ignore_debug_messages = True
env.ignore_layer_info_messages = True


In [ ]:
# Load the trace file if not sent from command-line
reload_helpers()

display(Markdown("""
## Loading GAPID trace: 
"""))

trace = None
if trace_file != None:
    display(Markdown(f"""
GAPID Trace: {trace_file}
    """))
    trace = gapid_trace.load_trace(env, trace_file)
else:
    def change_file(chooser):
        global trace
        current_trace = os.path.join(chooser.selected_path, chooser.selected_filename)
        display(Markdown(f"""
GAPID Trace: {current_trace}
    """))
        trace =  gapid_trace.load_trace(env, current_trace)
    
    fc = FileChooser()
    if env.current_trace:
        fc.default_path = os.path.dirname(env.current_trace)
        fc.default_filename = os.path.basename(env.current_trace)    
    fc.filter_pattern = "*.trace"
    fc.title = "<b>Select Trace</b>"
    fc.register_callback(change_file)
    display(fc)

## High-level parameters

In [ ]:
# Some trace stats
reload_helpers()

frames = trace.get_rendering_info()

print(f"Total frames in trace: {len(frames)}")
print(f"    Total renderpasses in trace: {sum([len(x) for x in frames])}")
print(f"    Total drawcalls in trace: {sum([sum([len(y.draw_calls) for y in x]) for x in frames])}")


## Renderpass Analysis of first frame

### Most Expensive RenderPasses

These values are taken across 10 sequential runs to help account for variability in hardware

In [ ]:
reload_helpers()

frametimes = []
all_timing_data = []

for x in range(10):
    timestamps = replay.timestamp_helper(env, trace, frames[0], silent=True)
    frametimes.append([x['end_time'] - x['start_time'] for x in timestamps])
    all_timing_data.append(timestamps)

top_render_passes = sorted(sorted(range(len(frametimes[0])), key=lambda i: frametimes[0][i])[-10:])
frametimes = numpy.asarray(frametimes)
top_frame_times = frametimes[:, top_render_passes]
fig = plt.figure()
ax = ax=fig.add_subplot(111)
bg = ax.boxplot(top_frame_times)
ax.set_title('Top 10 Renderpasses by render time')
ax.set_xlabel('Render Pass in frame')
ax.set_ylabel('Time in nanoseconds')

ax.set_xticklabels(top_render_passes)
plt.show()

### Breakdown of the 10 most expensive RenderPasses by draw-call

In [ ]:
reload_helpers()

most_expensive_renderpassses = list(numpy.asarray(frames[0])[top_render_passes])

draw_timestamps = replay.timestamp_helper(env, trace, most_expensive_renderpassses, silent=True, draw_calls=True)

grouped_times = []
for y in draw_timestamps:
    grouped_times.append([x['end_time'] - x['start_time'] for x in y['draws']])

biggest_value = len(max(grouped_times, key=lambda x: len(x)))
added_zeroes = [[0] * (biggest_value - len(x)) for x in grouped_times]
for x in range(len(added_zeroes)):
    grouped_times[x].extend(added_zeroes[x])
grouped_times = numpy.asarray(grouped_times).transpose()

fig = plt.figure()
ax = fig.add_subplot(111)
labels = [str(x) for x in top_render_passes]
bott = [0] * len(grouped_times[0])
for gt in grouped_times:
    ax.bar(labels, gt, bottom=bott)
    bott = [gt[i] + bott[i] for i in range(len(gt))]
ax.set_title('Top 10 Renderpasses draw time')
ax.set_xlabel('Render Pass in frame')
ax.set_ylabel('Time in nanoseconds')
plt.show()

## DrawCall Analysis of first frame

### Most Expensive DrawCalls

In [ ]:
reload_helpers()

frametimes = []
all_timing_data = []

for x in range(5):
    frame_draw_times = []
    timestamps = replay.timestamp_helper(env, trace, frames[0], silent=True, draw_calls=True)
    ts = []
    for i in range(len(timestamps)):
        if not timestamps[i]['draws']:
            continue
        for y in timestamps[i]['draws']:
            y['frame_time'] = y['end_time'] - y['start_time']
            y['submit_index'] = timestamps[i]['submit_index']
            y['command_buffer_index'] = timestamps[i]['command_buffer_index']
            y['render_pass_index'] = timestamps[i]['render_pass_index']
            ts.append(y)
    frametimes.append(ts)


mean_frametime = [sum([frametimes[i][j]['frame_time'] for i in range(len(frametimes))]) / float(len(frametimes)) for j in range(len(frametimes[0]))]

rps = sorted(range(len(frametimes[0])), key = lambda z: mean_frametime[z])
rps.reverse()


In [ ]:
sorted_frametimes = ([frametimes[0][x] for x in rps])

txts = []

for i in range(10):
    x = sorted_frametimes[i]
    submissions = [y for y in frames[0] if y.queue_submit == x["submit_index"] and y.queue_submission_index == x["command_buffer_index"]]
    nv = []
    nv.extend(['frame_time', str(x['frame_time'])])
    for k, v in trace.commands[submissions[x["render_pass_index"]].draw_calls[x["draw_index"]].recording_idx].items():
        if (k == "tracer_flags"):
            continue
        nv.extend([str(k), str(v)])
    txts.append(nv)
biggest_value = len(max(txts, key=lambda x: len(x)))
added_empties = [[""] * (biggest_value - len(x)) for x in txts]
for x in range(len(added_empties)):
    txts[x].extend(added_empties[x])

s = "|   " * (len(txts[0]) + 1) + "|\n"
s += "|-----"*(len(txts[0]) + 1)
s += "|\n"
s += "\n".join(["| " + " | ".join(x) + " |" for x in txts])
display(Markdown(s))

In [ ]:

draws_to_get = []
for i in range(10):
    x = sorted_frametimes[i]
    submissions = [y for y in frames[0] if y.queue_submit == x["submit_index"] and y.queue_submission_index == x["command_buffer_index"]]
    
    draws_to_get.append(SimpleNamespace(
        queue_submit=x["submit_index"],
        queue_submission_index=x["command_buffer_index"],
        idx=submissions[x["render_pass_index"]].draw_calls[x["draw_index"]].idx,
        rpidx=x["render_pass_index"]
    ))
draws_to_get.sort(key=lambda x: (x.queue_submit, x.queue_submission_index, x.idx))
fbs = replay.screenshot_helper(env, trace, draws_to_get, 1, before=True, after=True, silent=True)

### Drawing content of the 10 most expensive draw calls

In [ ]:
def normalize_image(img):
    if img.dtype == numpy.uint8:
        return img
    img = numpy.abs(img)
    nmax = numpy.max(img)
    if nmax == 0:
        return img.astype(numpy.uint8)
    img = (img.astype(numpy.double) * (255.0 / nmax)).astype(numpy.uint8)
    return img

width=32
height=(32/3 * len(fbs)/2)
f, grid = plt.subplots(int(len(fbs)/2), 3, figsize=(width, height))
for x in range(int(len(fbs)/2)):
    dtg = draws_to_get[x]
    before = normalize_image(numpy.flip(fbs[2*x], 0))
    after = normalize_image(numpy.flip(fbs[2*x+1], 0))
    diff = normalize_image(numpy.flip(numpy.abs(fbs[2*x] - fbs[2*x+1]), 0))
    grid[x, 0].imshow(before)
    grid[x, 0].title.set_text(f"Before Command {dtg.queue_submit}-{dtg.rpidx}-{dtg.idx}")
    grid[x, 1].imshow(after)
    grid[x, 1].title.set_text(f"After Command {dtg.queue_submit}-{dtg.rpidx}-{dtg.idx}")
    grid[x, 2].imshow(diff)
    grid[x, 2].title.set_text(f"Diff {dtg.queue_submit}-{dtg.rpidx}-{dtg.idx}")

plt.show()
    